In [1]:
import warnings
import string
import joblib
import multiprocessing
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import BertTokenizer
from transformers import BertModel
from torch import nn
from torch.nn import functional as F
from model import SentenceBert

warnings.filterwarnings("ignore")

I0708 15:54:01.107158  9796 file_utils.py:39] PyTorch version 1.2.0 available.


In [2]:
df = joblib.load("../data/reuters_news.joblib")
df = df.drop_duplicates(subset="title")
labels = ['forex', 'finance', 'stocks']
SB = SentenceBert()
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    sim_dict = SB.get_similarity(row["title"], labels)
    for i in range(len(labels)):   
        df.loc[index, labels[i]] = sim_dict[labels[i]]
df = df.sort_values(by="finance", axis=0, ascending=False)
df = df.reset_index(drop=True)
df.head()

I0708 15:54:04.062693  9796 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/deepset/sentence_bert/config.json from cache at C:\Users\YangWang/.cache\torch\transformers\1a89cc2d2dfc0e9beb7d49442e942849b45bbbf49d0b004f6be414b44c4e01fa.f9ed8a8332fc340fb779f9e83f1745369bbe51a3ac3e1c0d0dc5c3cf72ef4626
I0708 15:54:04.066643  9796 configuration_utils.py:321] Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0708 15:54:04.068637  9796 tokenization_utils.py:929] Model name 'deepset/sentence_bert' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base

,title,date,query,forex,finance,stocks
0,"Beware of debt costs and an inflationary bite,...",2020-07-05 19:17:00-08:00,google,0.064136,0.498809,0.287072
1,Deals of the day-Mergers and acquisitions,2020-07-07 16:00:00-08:00,google,0.157465,0.358413,0.328640
2,Zoom rolls out hardware subscription service,2020-07-07 10:48:00-08:00,google,0.101178,0.283011,0.171321
3,"In Hong Kong national security law, echoes of ...",2020-07-07 08:47:00-08:00,google,0.074657,0.224370,0.027718
4,China converts Hong Kong hotel into new nation...,2020-07-08 01:12:00-08:00,google,0.122923,0.223557,0.051019


In [3]:
PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

I0708 15:54:16.824857  9796 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\YangWang/.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0708 15:54:16.828886  9796 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

I0708 15:54:17.702512  9796 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface

In [4]:
sample = tokenizer.encode_plus(
    df["title"][0], 
    max_length=32, 
    add_special_tokens=True, 
    return_token_type_ids=False, 
    pad_to_max_length=True, 
    return_attention_mask=True, 
    return_tensors="pt")

In [8]:
class ReutersClassifier(nn.Module):
    def __init__(self, n_classes, p=0.25):
        super(ReutersClassifier, self).__init__()
        self.bert_layer = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.dropout = nn.Dropout(p=p)
        self.classifier = nn.Linear(self.bert_layer.config.dim, n_classes)
        
    def forward(self, input_ids, attention_mask):
        pooled_output = self.bert_layer(
            input_ids=input_ids, 
            attention_mask=attention_mask)
        main = self.dropout(pooled_output[0][:, 0, :])
        return F.sigmoid(self.classifier(main))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = ReutersClassifier(n_classes=1)
model.to(device)
print("Model built!")

I0708 15:55:32.213019  9796 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\YangWang/.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0708 15:55:32.216011  9796 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

I0708 15:55:32.345652  9796 modeling_utils.py:650] loading weights file https://cdn.huggingface.co/distilbert-ba

Model built!


In [9]:
probabilty = model(sample["input_ids"].to(device), sample["attention_mask"].to(device))

In [10]:
probabilty

tensor([[0.6039]], device='cuda:0', grad_fn=<SigmoidBackward>)